In [2]:
import pandas as pd

In [8]:
working_df = pd.read_csv("cleaned_car_accidents.csv")
working_df.head()

,Accident_Index,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),...,Vehicle_Reference_y,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Car_Passenger,Casualty_Type,Casualty_Home_Area_Type
0,200520D002602,-1.832342,52.534986,2,1,3,01/01/2005,7,20:23,300,...,1,1,1,1,-1,-1,3,0,9,1
1,200520D002703,-1.846293,52.465961,3,2,2,02/01/2005,1,18:45,300,...,2,1,1,1,31,6,3,0,9,1
2,200520D004203,-1.794089,52.455003,3,1,1,03/01/2005,2,17:04,300,...,1,1,1,1,24,5,3,0,9,1
3,200520D004603,-1.869954,52.478755,1,1,2,05/01/2005,4,15:05,300,...,1,1,3,1,31,6,1,0,0,1
4,200520D006503,-1.857470,52.469211,3,2,1,05/01/2005,4,22:00,300,...,2,1,1,2,22,5,3,0,9,1
